In [ ]:

import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd

#geo coordinate
def my_point(a,b):      
    return [float(a),float(b)]
#jaccard(did not use)
def jaccard_sim(a, b):                                         
    unions = len(set(a).union(set(b)))
    intersections = len(set(a).intersection(set(b)))
    return 1. * intersections / unions


#distance calculation    
def distance_between_points(point1,point2):
    return ((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)**0.5

list_raw_data_subway = []
subway_cord = []
#subway stations
with open('C:/Users/Administrator/Desktop/工作/POI/CSV版本/交通设施服务.csv') as raw_data_subway:
    raw_data_subway_csv = csv.reader(raw_data_subway)
    headers = next(raw_data_subway_csv)
    for row in raw_data_subway_csv:
        list_mid_subway = []       
        if row[3] == '地铁站':
            #print(row[0])
            subway_cord.append(my_point(row[8],row[9]))
            list_mid_subway.append(row[0])
            list_mid_subway.append(my_point(row[8],row[9]))
            list_raw_data_subway.append(list_mid_subway)
        
#schools
list_raw_data_edu = []
with open('C:/Users/Administrator/Desktop/工作/POI/CSV版本/edu.csv') as raw_data_edu:         
    raw_data_edu_csv = csv.reader(raw_data_edu)
    headers = next(raw_data_edu_csv)
    for row in raw_data_edu_csv:
        list_mid_edu = []
        list_mid_edu.append(row[7])
        list_mid_edu.append(row[0])
        list_mid_edu.append(row[1])
        list_mid_edu.append(row[2])
        list_mid_edu.append(row[3])
        list_mid_edu.append(jaccard_sim('学校',row[0]))
        list_mid_edu.append(my_point(row[8],row[9]))
        if list_mid_edu[1] != 0.0:
            list_raw_data_edu.append(list_mid_edu)

final_data = []


for location in list_raw_data_edu:
    for station_region in subway_stations:
        if isPoiWithinPoly(location[2],station_region[2]):            
            location.append(station_region[0])       
            location.append(distance_between_points(location[2],station_region[1]))     
            final_data.append(location)
            
            
            
            
print(final_data)


#generate binary variable
with open('C:/Users/Administrator/Desktop/工作/POI/CSV版本/trafficp r.csv') as raw_data_edu:         
    raw_data_edu_csv = csv.reader(raw_data_edu)
    headers = next(raw_data_edu_csv)
    for row in raw_data_edu_csv:
        list_mid_edu = []
        list_mid_edu.append(row[0])
        list_mid_edu.append(row[7])        
        list_mid_edu.append(row[1])
        list_mid_edu.append(row[2])
        list_mid_edu.append(row[3])
        if row[3] == '小学':
            list_mid_edu.append('1')
        else:
            list_mid_edu.append('0')
        if row[3] == '中学':
            list_mid_edu.append('1')
        else:
            list_mid_edu.append('0')
        list_mid_edu.append(jaccard_sim('学校',row[0]))
        list_mid_edu.append(row[8])
        list_mid_edu.append(row[9])
        if list_mid_edu[7] != 0.0:
            list_raw_data_edu.append(list_mid_edu)

#print(list_raw_data_edu)

#calculate the nearest distance
output_data = []

for school in list_raw_data_edu:
    distance_s = 100
    distance_b = 100
    distance_p = 100
    distance_c = 100    
    for s_station in list_raw_data_subway:
        if distance_between_points(my_point(school[8],school[9]),s_station[1]) < distance_s:
            distance_s = distance_between_points(my_point(school[8],school[9]),s_station[1])
    school.append(float(distance_s)*85)   
    for b_station in list_raw_data_bus:
        if distance_between_points(my_point(school[8],school[9]),b_station[1]) < distance_b:
            distance_b = distance_between_points(my_point(school[8],school[9]),b_station[1])
    school.append(float(distance_b)*85)    
    for p_lot in list_raw_data_p_lot:
        if distance_between_points(my_point(school[8],school[9]),p_lot[1]) < distance_p:
            distance_p = distance_between_points(my_point(school[8],school[9]),p_lot[1])
    school.append(float(distance_p)*85)    
    for c_station in list_raw_data_coach:
        if distance_between_points(my_point(school[8],school[9]),c_station[1]) < distance_c:
            distance_c = distance_between_points(my_point(school[8],school[9]),c_station[1])
    school.append(float(distance_c)*85)
    output_data.append(school)

print(output_data)


#output the data
head = [
    'name',
    'district',
    'kind_1',
    'kind_2',
    'kind_3',
    'is_primary_school',
    'is_middle_school',
    'Jaccard_sim',
    'longitude',
    'latitude',
    'distance_to_subway_station',
    'distance_to_bus_station',
    'distance_to_parking_lot',
    'distance_to_coach_station'
    ]

df = pd.DataFrame(output_data, columns=head)
print(df)
df.to_excel('C:/Users/Administrator/Desktop/output_data.xlsx')

